In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
import keras

In [ ]:
# GPU 장치 목록 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow는 GPU를 사용할 수 있습니다.")
    for gpu in gpus:
        print(f"사용 가능한 GPU: {gpu}")

    # gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
    # print(gpus)
    # if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)
else:
    print("TensorFlow는 GPU를 사용할 수 없습니다.")

In [ ]:
# EfficientNetB0 모델 로드 (ImageNet으로 사전 훈련된 가중치 사용)
model = keras.applications.EfficientNetB1(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

# 모델 요약 출력
# model.summary()

In [ ]:
import numpy as np
from PIL import Image

# 이미지 로드 및 전처리
def prepare_image(img_path, target_size=(224, 224)):
    # 이미지를 열고 리사이즈
    img = Image.open(img_path)
    img = img.resize(target_size)
    
    # 이미지를 numpy 배열로 변환하고 차원 추가
    img_array = np.array(img)
    
    # 색상 채널이 3개인지 확인
    if img_array.shape[-1] != 3:
        img_array = np.stack([img_array] * 3, axis=-1)
    
    img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가
    img_array = img_array.astype('float32') / 255.0  # 정규화
    return img_array

# 예측 함수
def predict_image(model, img_path):
    img_array = prepare_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class[0], predictions


In [ ]:

# 예측할 이미지 경로
img_path = 'image/horse.png'

# 예측 수행
predicted_class, predictions = predict_image(model, img_path)

In [ ]:
# ImageNet 클래스 이름 로드
decoded_predictions = keras.applications.efficientnet.decode_predictions(predictions, top=5)[0]

In [ ]:
# 특정 클래스 ID의 이름을 출력
for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
    print(f"{i + 1}: {label} ({score:.2f})")